<a href="https://colab.research.google.com/github/ojetokun/electricity_stats/blob/master/Hamoye_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df = df.drop(columns="stab")
df.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,unstable


In [75]:
X = df.drop(columns='stabf')
y = df['stabf']
#split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.2,random_state=1)

In [76]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)


In [22]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_x_train = scaler.fit_transform(x_train)
normalised_x_train = pd.DataFrame(normalised_x_train,columns=x_train.columns)

normalised_x_test = scaler.transform(x_test)
normalised_x_test = pd.DataFrame(normalised_x_test,columns=x_test.columns)
print(normalised_x_test.shape)
x_train

(2000, 12)


,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
2694,6.255995,2.542401,7.024714,9.476518,3.529888,-1.224881,-0.688228,-1.616780,0.568221,0.618403,0.685739,0.660088
5140,5.070581,5.490253,8.075688,0.761075,4.220888,-1.280596,-1.902185,-1.038107,0.443515,0.097244,0.916955,0.129254
2568,1.220072,8.804028,3.874283,8.433949,3.614027,-1.039236,-0.953566,-1.621224,0.908353,0.923594,0.238881,0.660156
3671,7.498402,6.697603,8.798626,2.126236,3.134585,-1.581906,-0.589386,-0.963293,0.260826,0.899003,0.964752,0.600598
7427,7.074006,1.337511,6.100756,7.759156,2.526922,-0.922540,-0.632600,-0.971782,0.984580,0.716082,0.836928,0.165162
...,...,...,...,...,...,...,...,...,...,...,...,...
2895,9.504984,5.265415,2.029617,8.026334,3.449756,-0.922754,-1.526305,-1.000696,0.191864,0.611297,0.080008,0.350312
7813,8.035818,4.632548,1.177810,1.204950,4.093208,-1.568218,-0.819966,-1.705024,0.223384,0.317770,0.726623,0.441620
905,7.052560,3.264057,1.405949,4.497549,3.741358,-0.629026,-1.407945,-1.704387,0.868306,0.605566,0.100591,0.747320
5192,5.085242,1.790031,2.486392,2.848192,3.159670,-0.892898,-1.025461,-1.241311,0.473917,0.418512,0.326253,0.982488


In [54]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score,confusion_matrix

clf = ExtraTreesClassifier(random_state=1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
print(accuracy)


0.928


In [64]:
# with new params
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score,confusion_matrix

etc_new_clf = ExtraTreesClassifier(random_state=1,max_features=None,
                                          min_samples_leaf= 8,
                                          min_samples_split= 2,
                                          n_estimators=1000)
z= etc_new_clf.fit(x_train, y_train)
etc_new_clf_y_pred = etc_new_clf.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=etc_new_clf_y_pred)
print(accuracy)

0.927


In [71]:
importance = z.feature_importances_
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.13724
Feature: 1, Score: 0.14051
Feature: 2, Score: 0.13468
Feature: 3, Score: 0.13542
Feature: 4, Score: 0.00368
Feature: 5, Score: 0.00534
Feature: 6, Score: 0.00543
Feature: 7, Score: 0.00496
Feature: 8, Score: 0.10256
Feature: 9, Score: 0.10758
Feature: 10, Score: 0.11306
Feature: 11, Score: 0.10954


In [30]:
n_estimators = [50, 100, 300, 500, 1000]
min_samples_split = [2, 3, 5, 7, 9]
min_samples_leaf = [1, 2, 4, 6, 8]
max_features = ['auto', 'sqrt', 'log2', None]
hyperparameter_grid = {'n_estimators': n_estimators,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features}

In [53]:
from sklearn.model_selection import RandomizedSearchCV

etc_clf = ExtraTreesClassifier(random_state=1)
grid_clf = RandomizedSearchCV(etc_clf, hyperparameter_grid, cv=5, n_iter=10, scoring = 'accuracy', n_jobs = -1, verbose = 1,random_state = 1)
search = grid_clf.fit(x_train, y_train)
search.best_params_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.5min finished


{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [72]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=1)
forest_clf.fit(x_train, y_train)
forest_y_pred = forest_clf.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=forest_y_pred)
print(accuracy)

0.929


In [49]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(random_state=1)
xgb_clf.fit(x_train, y_train)
xgb_y_pred = xgb_clf.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=xgb_y_pred)
print(accuracy)

0.9195


In [52]:
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(random_state=1)
lgbm_clf.fit(x_train, y_train)
lgbm_y_pred = lgbm_clf.predict(x_test)
accuracy = accuracy_score(y_true=y_test, y_pred=lgbm_y_pred)
print(accuracy)

0.9375
